# 서론
---

## 주제: 실시간 숫자 손글씨 검출
참여 연구자: 김동주, 이도경

## 요약:
OpenCV를 이용하여 동영상으로 부터 손글씨를 검출한다.  
모델 학습에는 두 가지 방법을 사용한다.  

1. MNIST 데이터베이스를 이용하여 모델을 지도
2. 레이블 되어진 동영상을 이용하여 모델을 지도 (직접 획득한 데이터를 이용)
  
두 모델에 대한 탐구를 통하여 다음의 목표를 달성하고자 한다.

* 영상처리를 이용한 실시간 분류기에 대한 접근 및 이해  
* 향후 연구방향 제시

# 본론
---

## 목표: 동영상을 이용한 Machine Learning 지도학습 데이터 수집

## 설계:
본 프로그램의 구현 과정은 크게 3가지 단계로 이루어집니다.

1. raw 데이터 획득
    * 균일한 색상의 배경과 특정색상 선을 사용하여 배경과 검출하고자 하는 숫자를 명확하게 함. (화이트보드/붉은색 마커 사용)
    * 녹화된 영상의 파일명에 해당 숫자 값을 명시하여 레이블함. (ex: `<정답>_<영상번호>.mov`)

2. 데이터셋 준비
    * OpenCV를 이용하여 데이터를 28x28, Single-channel Image로 통일시킴
    * 데이터에 올바른 레이블을 매칭시켜줌.

3. 머신러닝 모델에 적용
    * TensorFlow 시스템을 이용하여 모델 구현, 학습, 그리고 검증

---
    
## 1. raw 데이터 획득

가장 먼저 해야할 일은 역시 데이터 수집입니다.  
힘들지만 손수 글씨를 써가며 촬영하는 방식으로 데이터를 획득하였습니다.

촬영한 영상 원본은 다음 링크에서 확인하실 수 있습니다.

* Google Drive URL: https://drive.google.com/drive/folders/1-HyJBGEiGAk_lxCEdQ6mXYBZVoAfACZt

---
## 2. 데이터셋 준비

### 2.1 데이터 정규화

다음의 코드를 이용하여 정규화(: 화면 자르기, 손글씨 검출, 노이즈제거, 차원축소, 크기축소) 작업을 수행합니다.

In [ ]:
# -----------------------------------------------------------
# this code normalizes raw video files into 28x28 single-channel images
# 
# (C) 2020 Kim Dong Joo, Dongguk University, Gyeongju
# email hepheir@gmail.com
# -----------------------------------------------------------

import cv2
import numpy as np
import os

RAW_DATA_PATH = 'res/raw/'
OUT_DATA_PATH = 'res/out/'

def main():
    ls = os.listdir(RAW_DATA_PATH)

    usr_sel = input("%d files found.\nproceed? [y/n]: " % len(ls))
    if usr_sel != 'y':
        print("Canceled by user.")
        return

    # -----------------------------------------------------------

    for file_index in range(len(ls)):
        filename = ls[file_index]

        if filename.startswith('TEST'): continue

        video = cv2.VideoCapture(RAW_DATA_PATH + filename)
        frame_index = 0
        
        while video.isOpened():
            ret, frame = video.read()
            if not ret: break

            # -----------------------------------------------------------
            # Cut & Resize
            #   * why [:,420:-420]?
            #   --> frame.shape of the used videos are 1920x1080.
            #       to make it square, sliced 'x' from each frames from 420 to -420
            #       (1080, 1920, 3) -> (1080, 1080, 3)
            #   * why resize to (488,488)?
            #   --> to fasten the process.
            #       chose 448, since it is 28 multiplied by 2^4
            #       (448=28*(2^4))
            # -----------------------------------------------------------
            frame = frame[::-1,-420:420:-1] 
            frame = cv2.resize(frame, (448,448))

            # -----------------------------------------------------------
            # Detect hand writings
            # -----------------------------------------------------------
            yuv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
            red_yuv_range = (( 64,  0,128),
                             (255,128,255))
            red_yuv_mask = cv2.inRange(yuv_frame, red_yuv_range[0], red_yuv_range[1])

            hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            red_hsv_range = ((  0, 64, 64),
                             (255,255,255))
            red_hsv_mask = cv2.inRange(hsv_frame, red_hsv_range[0], red_hsv_range[1])
            
            red_mask = cv2.bitwise_and(red_yuv_mask, red_hsv_mask)

            # -----------------------------------------------------------
            # Dilate mask to make them survive from resizing
            # -----------------------------------------------------------
            red_mask = cv2.dilate(red_mask,
                                  cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9)),
                                  iterations=3)

            # -----------------------------------------------------------
            # Resize and export frames
            # -----------------------------------------------------------
            out_frame = cv2.resize(red_mask, (28, 28))
            out_filename = OUT_DATA_PATH + filename.replace('.MOV', '_%d.png' % frame_index)

            cv2.imwrite(out_filename, out_frame)
            frame_index += 1

            print("\rCreating (%d/%d)... %s         " % (file_index+1, len(ls), out_filename), end="")
        video.release()

if __name__ == '__main__':
    main()
    print("\nDone.")

### 2.2 레이블 매칭

다음은 벤치마킹할 mnist의 손글씨 검출 학습데이터입니다.
이 예제를 통해 데이터셋이 어떠한 형태로 저장되는지 알아볼 수 있습니다.

In [ ]:
import tensorflow as tf
import cv2

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('x_train[0]\n', x_train[0])
print('y_train[0]\n', y_train[0])

cv2.imshow(y_train[0], x_train[0])

cv2.waitKey(0)
cv2.destroyAllWindows()

앞선 예제를 통해 데이터셋이 어떠한 형태로 주어지는지를 보았습니다.
`x_train`, `y_train`에는 각각 (28x28xN), (label x N)의 형태로 데이터와 레이블이 존재하며 1:1 대응을 이루게 됩니다.
  
이를 바탕으로 '2. 데이터 정규화'에서 획득한 이미지들의 레이블링을 수행합니다.

In [ ]:
import os
import cv2
import numpy as np

DATA_PATH = 'res/out/'

x_train, y_train = [], []

for filename in os.listdir(DATA_PATH):
    if filename.startswith('TEST'): continue # 레이블링이 불가능한 영상은 제외

    frame = cv2.imread(DATA_PATH + filename)
    label = filename.split('_')[0]
    
    x_train.append(frame)
    y_train.append(label)

x_train = np.array(x_train, dtype=np.uint8)
y_train = np.array(y_train, dtype=np.uint8)

---
##  3. 실제 모델에 적용

이번에는 mnist 데이터셋을 이용하여 학습시킨 모델에 사용자 데이터셋('3. 레이블 매칭'에서 생성)을 적용시켜 볼 것입니다.  

### 3.1 mnist 데이터셋으로 학습시킨 모델
우선은 mnist 데이터셋을 이용하여 모델을 학습시키고 결과를 적용합니다. [모델 저장과 복원 참고](https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko)

In [26]:
import tensorflow as tf

checkpoint = 'models/checkpoints/mnist_tmp'

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test, verbose=2)

model.save_weights(checkpoint)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.2949 - accuracy: 0.9141
Epoch 2/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.1415 - accuracy: 0.9578
Epoch 3/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.1078 - accuracy: 0.9671
Epoch 4/5
60000/60000 [==============================] - 6s 105us/sample - loss: 0.0898 - accuracy: 0.9717
Epoch 5/5
60000/60000 [==============================] - 5s 87us/sample - loss: 0.0748 - accuracy: 0.9764
10000/10000 - 0s - loss: 0.0732 - accuracy: 0.9778


### 3.2 저장한 모델을 불러오기
다음 코드를 통해 모델이 제대로 저장되었는지 확인해봅니다.

In [60]:
import tensorflow as tf

from models import mnist

model = mnist.model # 3.1 에서 사용한 모델

model.load_weights(mnist.checkpoint)
model.evaluate(x_test, y_test, verbose=2)

3281/3281 - 0s - loss: 12.7142 - accuracy: 0.1439


[12.71417108783995, 0.14385858]

### 3.3 사용자 데이터셋으로 테스트
모델이 제대로 저장되었음을 확인했으니, 모델에 준비해온 데이터셋을 적용해보겠습니다.

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf

from models.mnist import model, checkpoint

DATA_PATH = './res/out/'

def load_user_data():
    x, y = [], []
    for filename in os.listdir(DATA_PATH):
        if filename.startswith('TEST'): continue
        
        frame = cv2.imread(DATA_PATH + filename, cv2.IMREAD_GRAYSCALE)
        label = filename.split('_')[0]
        
        if type(frame) is type(None): continue # To sort out dummy files as '.DS_Store'
        if not frame.any(): continue # Empty frame

        x.append(frame.copy())
        y.append(label)

    x = np.array(x, dtype=np.uint8)
    y = np.array(y, dtype=np.uint8)
    return x/255.0, y

if __name__ == '__main__':
    x_test, y_test = load_user_data()

    model.load_weights(checkpoint)
    model.evaluate(x_test, y_test)


3281/3281 [==============================] - 0s 105us/sample - loss: 17.1309 - accuracy: 0.1344


정말 아쉽게도 14.39%의 정확성밖에 보이지 못하였습니다.  
  
아마도 사용자 데이터 중에서 글씨가 덜 써진, 혹은 기타 노이즈(ex. 글씨외에도 손이나 팔뚝이 검출)로 인하여 올바르게 레이블이 되어지지 않은 프레임들이 문제가 되지 않나 싶습니다.

## 4 실시간 손글씨 검출

보다 직관적인 결과 확인을 위해, 동영상에서 실시간으로 손글씨 분류 결과를 확인 할 수 있는 모델을 생성해보기로 합니다.  
위 모델에서 샘플당 처리소요시간은 74us로, 실시간 처리에는 큰 지장이 없으리라 생각이됩니다.

### 4.1 카메라 이용 및 녹화

우선은 카메라를 이용하는 방법과 영상을 녹화하는 방법에 대하여 간단히 훑어봅니다.

In [ ]:
# -----------------------------------------------------------
# demonstrate how to record a video with a camera using opencv-python
# 
# (C) 2020 Kim Dong Joo, Dongguk University, Gyeongju
# email hepheir@gmail.com
# -----------------------------------------------------------

import cv2
from datetime import datetime

# -----------------------------------------------------------
# Video I/O Settings
# -----------------------------------------------------------

FRAME_WIDTH, FRAME_HEIGHT = 256, 256
FPS = 20.0

# 녹화된 영상이 저장될 곳
FILE_NAME_PATTERN = "res/records/"+"base-%y%m%d_%H%M%S.avi"


# -----------------------------------------------------------
# Utilities
# -----------------------------------------------------------

status = {
    'frames': 0,
    'video_length': 0}

def updateStatus():
    # Put all the values to be updated for each cycles
    global status
    status['frames'] += 1
    status['video_length'] = status['frames'] // FPS

def showStatus():
    global status
    msg = "\r"
    for key in status:
        msg += "%s: %s " % (key, str(status[key]))
    print(msg, end="")

# -----------------------------------------------------------
# Main loop
# -----------------------------------------------------------
if __name__ == '__main__':
    vidIn = cv2.VideoCapture(0)
    vidIn.set(cv2.CAP_PROP_FRAME_WIDTH,  FRAME_WIDTH)
    vidIn.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    vidOut = cv2.VideoWriter(datetime.now().strftime(FILE_NAME_PATTERN),
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             FPS,
                             (FRAME_WIDTH, FRAME_HEIGHT))

    while vidIn.isOpened():
        ret, frame = vidIn.read()
        if not ret: break

        frame = cv2.resize(frame, (FRAME_HEIGHT, FRAME_WIDTH))
        # -----------------------------------------------------------
        # TODO
        # -----------------------------------------------------------
        cv2.imshow('cam', frame)
        vidOut.write(frame)
        
        # -----------------------------------------------------------
        # Key mappings
        # -----------------------------------------------------------
        key = cv2.waitKey(20) & 0xFF
        if key == 27: break # ESC

        # -----------------------------------------------------------
        # Debugs
        # -----------------------------------------------------------
        updateStatus()
        showStatus()

    vidIn.release()
    vidOut.release()
    cv2.destroyAllWindows()

print("")
print("Successfully Closed")

---

### 4.2 실시간 예측 모델

다음은 카메라 장치를 이용하여 영상을 받아오고, 실시간으로 손글씨를 검출, 정규화하는 스크립트입니다. 
학습된 모델을 사용하여 정규화한 데이터의 값을 예측합니다.  
  
만약 모델이 만족스럽지 못한 예측을 할 때를 대비하여, 재훈련 기능도 넣었습니다.

In [2]:
# -----------------------------------------------------------
# demonstrate how to classify hand writings from a video stream
# being captured in real-time, using opencv-python
# 
# (C) 2020 Kim Dong Joo, Dongguk University, Gyeongju
# email hepheir@gmail.com
# -----------------------------------------------------------

import cv2
import numpy as np
import tensorflow as tf

# 모델을 바꾸려면 여기서 수정.
from models.mnist import model, checkpoint, input_shape

def writeText(frame, txt):
    frame[:28,:] = frame[:28,:] // 2
    cv2.putText(frame,
                txt,
                (32, 16),               # Coordinates
                cv2.FONT_HERSHEY_PLAIN, # 
                1.2,                    # Font scale
                (128, 128, 255),        # Font color
                lineType=cv2.LINE_AA)

def getNumericKey(key):
    # 0~9사이의 키가 눌렸을 때만 해당 키를 반환.
    # 그 외의 키가 눌리면 None을 반환한다.
    c = chr(key)
    if c in "0123456789":
        return int(c)
    return None

if __name__ == '__main__':
    model.load_weights(checkpoint)

    # Re-train settings
    if input("Load weights?[y/n] :") == 'y':
        model.load_weights(checkpoint+'_tmp')
    doSave = input("Save weights?[y/n] :") == 'y'

    FRAME_WIDTH, FRAME_HEIGHT = 448, 448
    # vidIn = cv2.VideoCapture(0)
    vidIn = cv2.VideoCapture('resources/raw/TEST.MOV')

    # ===========================================================
    # Main loop
    # ===========================================================
    while vidIn.isOpened():
        ret, frame = vidIn.read()
        if not ret: break

        frame = frame[:,420:-420] # frame[::-1,-420:420:-1]
        frame = cv2.resize(frame, (FRAME_HEIGHT, FRAME_WIDTH))
        
        frame = cv2.flip(frame, 0) # flip virtically (up and down)
        frame = cv2.flip(frame, 1) # flip horizontally (left and right)

        # -----------------------------------------------------------
        # Detect hand writings & normalize
        # -----------------------------------------------------------
        yuv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)
        red_yuv_mask = cv2.inRange(yuv_frame,
                                   ( 64,  0,128),
                                   (255,128,255))

        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        red_hsv_mask = cv2.inRange(hsv_frame,
                                   (  0, 48, 48),
                                   (255,255,255))
        red_mask = cv2.dilate(cv2.bitwise_and(red_yuv_mask, red_hsv_mask),
                              cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9)),
                              iterations=3)
        normalized_mask = cv2.resize(red_mask,(28,28))

        # -----------------------------------------------------------
        # Predict using pre-trained model
        # -----------------------------------------------------------s
        x = np.expand_dims(normalized_mask.reshape(input_shape), axis=0)
        p = model.predict(x)[0] # predicts 1 case only.

        y = np.where(p == max(p))[0][0] # 가끔 p가 one-hot encoded가 아닌 잡음이 섞여나오는 문제를 해결

        # -----------------------------------------------------------
        # Key mappings
        # -----------------------------------------------------------
        key = cv2.waitKey(20) & 0xFF
        if key == 27: break # ESC

        # -----------------------------------------------------------
        # Re-train model (if the prediction is wrong)
        # -----------------------------------------------------------
        x_train = x
        y_train = np.expand_dims(getNumericKey(key), axis=0)

        if y_train[0] != None: # 레이블이 주어지면 학습
            model.fit(x_train, y_train, epochs=1)

        # -----------------------------------------------------------
        # Show the result on screen
        # -----------------------------------------------------------
        writeText(frame, "Predict: %d, Label : %s" % (y, str(y_train[0])))
        frame[:28,:28] = x[0]
        cv2.imshow('Camera', frame)

    vidIn.release()
    cv2.destroyAllWindows()

    # Re-train on demand
    if doSave:
        model.save_weights(checkpoint+'_tmp')
        print("Weights saved.")

print("")
print("Successfully Closed")


Successfully Closed


# 결론
---

## 실험결과

손 글씨 검출 모델의 예측결과가 생각보다 좋지 않았습니다.  
  
글자가 제대로 인식되는 경우모다 엉뚱한 숫자가 예측되는 경우가 압도적으로 많았습니다.
  
  
  
## 낮은 정확도의 원인 추측
  
처음에는 낮은 정확도가 나오는 원인이 다음과 같지 않을까 생각하였습니다.  
  
* 모델의 사전 훈련이 충분치 않았다
* 모델이 너무 단순하다

### 모델의 사전 훈련이 충분치 않았다?
  
우선 모델의 사전 학습 단계에서의 epoch 수를 6 에서 12 이상으로 증가시켜 보았습니다.  
그러나 `models/mnist.py`의 성능은 97~98%대에서 머무르는 등 큰 성능의 변화를 기대할 수 없었습니다.    


### 모델이 너무 단순하다?

mnist 데이터 셋을 이용한 다양한 예제들 중에는 분명 `models/mnist.py`보다 더 복잡해보이는 모델도 많이 있었습니다.  
다양한 예제에서 사용되는 `Conv2D()` 층을 사용하여 모델을 설계해보기로 합니다.  
우선은 사용하는 모델을 2-layer CNN 모델로(`models/mnist_2_layers.py`)로 수정한 후, 학습을 진행해보기로 하였습니다.  
  
  
`epoch=8`에서 `model.evalutate()`을 이용한 성능평가시에는 98%의 중후반의 정확도를 유지했습니다. 조금 더 fitting 시의 epoch를 늘리면 정확도 99%로의 도달을 조금은 낙관적으로 기대해보았습니다...만, 아쉽게도 모델의 최종 평가 및 체크포인트 생성 과정에서 반복하여 프로그램 에러가 발생하였습니다.  
분명히 완벽하게 프로그래밍을 했다고 생각했는데 말이죠.

```shell
ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (10000, 28, 28)
```
  

## 실패 원인 분석
  
다음 두 가지 질문에 초점을 맞추어 문제를 해결해보기로 하였습니다.
* '낮은 정확도가 나오는 원인이 무엇인가'
* '내가 CNN 모델 층을 어떻게 이해하고 구현하고 있는가'

### 실패 원인 분석 중 알게 된 것

실패 원인을 분석, 모델을 개선하기 위해 알아보던 중, 다음과 같은 코멘트를 보게 되었습니다
> [정이교 @ 페이스북 커뮤니티::케라스코리아](https://www.facebook.com/groups/KerasKorea/3534408366575103/?notif_id=1588753428559249&notif_t=group_nf_highlights)  
> reshape로 보통 같은 이미지를 만들고 학습합니다. 
> 위에서 언급해주셨듯, crop이나 rotation, filp flop등을 사용해서 데이터 개수를 늘려서 정확도를 늘리는 학습을 시킵니다.
> data augmentation을 찾아보시면 될거에요. 추가로 gaussian noise를 넣기도 합니다.
> 다른 크기의 이미지를 학습시킬때에는, input을 두가지로 정의해주고 네트워크 안에서 dimension을 dense 등으로 맞춰주어야합니다.

또한 지난 세미나 발표들 중 오재호 연구원께서 소개해주셨던 선형변환에 강한 SIFT, SURF 알고리즘에 대한 것도 머릿속을 스쳐 지나갑니다. 

  

## * 향후 연구 계획

그리하여, 다음주에는 동일한 주제와 목표를 유지하면서, 성능향상을 위한 방법에 대한 학습을 해보려 합니다.  
조금은 부실했던, 머신러닝; 특히 CNN과 다양한 종류의 층, 모델을 어떻게 설계해야하는지 대한 부족했던 이해를 메꾸어 보려고 합니다.  
  

당장 생각하고 있는건, 손글씨 영상의 학습데이터 생성시, 선형변환을 고려한 데이터셋 생성 및 더 견고한(다층) 모델을 안정적으로 시험해 보는 것입니다.  
(좌우반전, 회전, 스케일링을 고의적으로 적용하여 하나의 이미지로부터 다양한 데이터셋을 만듬)

---

# 부록
---
## 참고
* Teachable Machine: 심태섭 연구원이 소개해준 온라인 머신러닝 서비스로, 본 활동에 영감을 받음
* 